In [1]:
import numpy as np
import pandas as pd
import pulp as lp
import pickle as pkl
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import ALMPlanner as ALM
import ALMChart as ALMc

In [2]:
######
EX = 1
######


if EX == 1:
    problem = ALM.ALMplanner(start = "Jan 2021", end = "Jan 2041", user_risk_profile = 0)
    # set planned liabilities
    problem.liabilities.insert("car", "Jan 2026", 30000, 30000*0.65)
    problem.liabilities.insert("university", "Jan 2029", 50000, 50000*0.95)
    problem.liabilities.insert("hawaii", "Jan 2037",30000, 30000*0.85)
    # set planned assets 
    problem.assets.insert("ass_0","Jan 2021",30000)
    recurrent_dates = ["Jan 2022", "Jan 2023", "Jan 2024", "Jan 2025", "Jan 2026", "Jan 2027"]
    for i in np.arange(len(recurrent_dates)):
        problem.assets.insert("ass_" + str(i+1),recurrent_dates[i],10000)
elif EX == 2:
    problem = ALM.ALMplanner(start = "Jan 2021", end = "Jan 2061")
    # set planned liabilities
    recurrent_assets = pd.date_range(start = "Jan 2021", end="Jan 2040", freq = pd.offsets.YearBegin(1))
    for i in np.arange(len(recurrent_assets)):
        problem.assets.insert("ass_" + str(i+1),recurrent_assets[i],1000)
    recurrent_liabilities = pd.date_range(start = "Jan 2041", end="Jan 2060", freq = pd.offsets.YearBegin(1))
    for i in np.arange(len(recurrent_liabilities)):
        problem.liabilities.insert("liab_" + str(i+1),recurrent_liabilities[i],1500,1100)

ALMc.display(problem,bar_width = 6)

In [3]:
problem.check_feasibility()
problem.get_feasibility()
ALMc.display(problem,bar_width = 6)

check feasibility ended in 13.18 s with Optimal solution
Unfeasible problem: suggested +2.53%  on assets


In [4]:
GB_model = ALM.ALMGoalBased(problem)
GB_model.solve(problem)

BaH_model = ALM.ALMBuyAndHold(problem)
BaH_model.solve(problem)

GoalBased model generated in 11.59 s
Solve ended in 2.05 s with Optimal solution
BuyAndHold model generated in 12.21 s
Solve ended in 0.95 s with Optimal solution


# Display solution

In [5]:
P = problem.P
A = problem.assets.set
L = problem.liabilities.set
N = problem.N
T = problem.T
Scenario = problem.Scenario

In [32]:
sol = GB_model.solution
#sol = BaH_model.solution


In [7]:
Assets_l = {}
Assets_end = {}
for a in A:
    Assets_l[a] = np.zeros(shape = (len(L), len(P)))
    Assets_end[a] = np.zeros(shape = (len(P)))
    for p in np.arange(len(P)):
        Assets_end[a][p] = sol.asset_to_exwealth[a][P[p]]
        for l in np.arange(len(L)):
            Assets_l[a][l,p] = sol.asset_to_goal[a][L[l]][P[p]]

In [9]:
#AssetPerETF
AssetPerETF = {}

for a in A:
    Asset_ap = np.sum(Assets_l[a], axis = 0) + Assets_end[a] 
    AssetPerETF[a] = {P[p]:Asset_ap[p] for p in np.arange(len(P))}

AssetPerETF = pd.DataFrame(AssetPerETF).transpose().reset_index()
AssetPerETF_melt = AssetPerETF.melt(id_vars=['index'], var_name='P', value_name='split')
plt = px.bar(AssetPerETF_melt, x="index", y = "split", color = "P" )
plt = ALMc.standardized_chart(plt)
#AssetPerETF.plot.bar(stacked = True, figsize = (15,10))
plt.show()

In [10]:
AssetPerLiab = {}
for a in A:
    Asset_al = list(np.sum(Assets_l[a], axis = 1))
    Asset_al.append(np.sum(Assets_end[a]))
    L_end = list(L)
    L_end.append("end")
    AssetPerLiab[a] = {L_end[l]:Asset_al[l] for l in np.arange(len(L_end))}

AssetPerLiab = pd.DataFrame(AssetPerLiab).transpose().reset_index()
print(AssetPerLiab.sum())
AssetPerLiab_melt = AssetPerLiab.melt(id_vars=['index'], var_name='Liab', value_name='split')
plt = px.bar(AssetPerLiab_melt, x="index", y = "split", color = "Liab")
plt = ALMc.standardized_chart(plt)
plt.show()

index         ass_0ass_1ass_2ass_3ass_4ass_5ass_6
car                                       19453.0
university                                47386.0
hawaii                                    25438.0
end                                       1.85094
dtype: object


In [14]:
n_scen = 5
#print([solution[l][n_scen].varValue for l in Lt])
Q_nscen = np.zeros(shape = (len(P),len(L)))
Val_tl = {}
i = -1
At = sorted(list(problem.assets.period.values()))
label_period = [[k, v] for k,v in problem.assets.period.items()]
label_value = [[k, v] for k,v in problem.assets.value.items()]
period_value_df = pd.DataFrame(label_period, columns = ["Label", "Period"]).merge(pd.DataFrame(label_value, columns = ["Label", "Value"]))

for p in P:
    i = i+1
    Val_tl[p] = np.zeros(shape = (len(T), len(L)))
    for l in np.arange(len(L)):
        for t in np.arange(len(T)):
            if t < problem.liabilities.period[L[l]]:
                new_asset_label = period_value_df.loc[period_value_df.Period == t , "Label" ].values
                new_asset = [sol.asset_to_goal[a][L[l]][p] for a in new_asset_label]
                if t==0:
                    Val_tl[p][t,l] = sum(new_asset)*np.exp(Scenario[p][n_scen][t])
                else:
                    Val_tl[p][t,l] = (Val_tl[p][t-1,l] + sum(new_asset))*np.exp(Scenario[p][n_scen][t])

            elif t == problem.liabilities.period[L[l]]:
                Q_nscen[i,l] = Val_tl[p][t-1,l]
    
Val_end_t = {}
for p in P:
    Val_end_t[p] = np.zeros(shape = (len(T)))
    for t in np.arange(len(T)):
        new_asset_label = period_value_df.loc[period_value_df.Period == t , "Label" ].values
        new_asset = [sol.asset_to_exwealth[a][p] for a in new_asset_label]
        if t==0:
            Val_end_t[p][t] = sum(new_asset)*np.exp(Scenario[p][n_scen][t])
        else:
            Val_end_t[p][t] = (Val_end_t[p][t-1] + sum(new_asset))*np.exp(Scenario[p][n_scen][t])


Val_t = {}
for p in P:
    Val_t[p] = np.sum(Val_tl[p], axis = 1) + Val_end_t[p] 

AssetAllocationNominal = pd.DataFrame(Val_t)
AssetAllocationNominal[AssetAllocationNominal<0] = 0
AAN_perc = AssetAllocationNominal.divide(AssetAllocationNominal.sum(axis=1), axis=0)

AssetAllocationNominal = AssetAllocationNominal.reset_index()
AssetAllocationNominal = AssetAllocationNominal.melt(id_vars=['index'], var_name='P', value_name='evo')
plt = px.area(AssetAllocationNominal, x="index", y = "evo", color = "P" )
plt = ALMc.standardized_chart(plt)
plt.show()

AAN_perc = AAN_perc.reset_index()
AAN_perc = AAN_perc.melt(id_vars=['index'], var_name='P', value_name='evo')
plt = px.area(AAN_perc, x="index", y = "evo", color = "P" )
plt.update_xaxes(range=[AAN_perc["index"].min(), AAN_perc["index"].max()])
plt.update_yaxes(range=[0, 1])
plt = ALMc.standardized_chart(plt)
plt.show()

In [23]:
Scenario_mu = problem.Scenario_mu
#print([solution[l][n_scen].varValue for l in Lt])
Q_nscen = np.zeros(shape = (len(P),len(L)))
Q_ex_tot = np.zeros(shape = (len(P),len(L)))
Val_tl = {}
index = -1
At = sorted(list(problem.assets.period.values()))
label_period = [[k, v] for k,v in problem.assets.period.items()]
label_value = [[k, v] for k,v in problem.assets.value.items()]
period_value_df = pd.DataFrame(label_period, columns = ["Label", "Period"]).merge(pd.DataFrame(label_value, columns = ["Label", "Value"]))

for p in P:
    index = index+1
    Val_tl[p] = np.zeros(shape = (len(T), len(L)))
    for l in np.arange(len(L)):
        for t in np.arange(len(T)):
            if t < problem.liabilities.period[L[l]]:
                new_asset_label = period_value_df.loc[period_value_df.Period == t , "Label" ].values
                new_asset = [sol.asset_to_goal[a][L[l]][p] for a in new_asset_label]
                if t==0:
                    Val_tl[p][t,l] = sum(new_asset)*np.exp(Scenario_mu[p])
                else:
                    Val_tl[p][t,l] = (Val_tl[p][t-1,l] + sum(new_asset))*np.exp(Scenario_mu[p])

            elif t == problem.liabilities.period[L[l]]:
                Q_nscen[index,l] = Val_tl[p][t-1,l]



Val_end_t = {}
index = -1
for p in P:
    index = index+1
    Val_end_t[p] = np.zeros(shape = (len(T)))
    for t in np.arange(len(T)):
        ex_wealth = 0
        for l in np.arange(len(L)):
            if t == problem.liabilities.period[L[l]]:
                ex_wealth = ex_wealth + max(sum(Q_nscen[:,l]) - problem.liabilities.value_tg[L[l]], 0)

        new_asset_label = period_value_df.loc[period_value_df.Period == t , "Label" ].values
        new_asset = [sol.asset_to_exwealth[a][p] for a in new_asset_label]
        ex_wealth_p = ex_wealth*problem.user_portfolio[p]
        if t==0:
            Val_end_t[p][t] = sum(new_asset)*np.exp(Scenario_mu[p])
        else:
            Val_end_t[p][t] = (Val_end_t[p][t-1] + sum(new_asset) + ex_wealth_p)*np.exp(Scenario_mu[p])


Val_t = {}
for p in P:
    Val_t[p] = np.sum(Val_tl[p], axis = 1) + Val_end_t[p] 

AssetAllocationNominal = pd.DataFrame(Val_t)
AssetAllocationNominal[AssetAllocationNominal<0] = 0
AAN_perc = AssetAllocationNominal.divide(AssetAllocationNominal.sum(axis=1), axis=0)

AssetAllocationNominal = AssetAllocationNominal.reset_index()
AssetAllocationNominal = AssetAllocationNominal.melt(id_vars=['index'], var_name='P', value_name='evo')
plt = px.area(AssetAllocationNominal, x="index", y = "evo", color = "P")
plt = ALMc.standardized_chart(plt)
plt.show()

AAN_perc = AAN_perc.reset_index()
AAN_perc = AAN_perc.melt(id_vars=['index'], var_name='P', value_name='evo')
plt = px.area(AAN_perc, x="index", y = "evo", color = "P" )
plt.update_xaxes(range=[AAN_perc["index"].min(), AAN_perc["index"].max()])
plt.update_yaxes(range=[0, 1])
plt = ALMc.standardized_chart(plt)
plt.show()

In [60]:
Q_ln = {}
for l in L:
    Q_ln[l] = np.zeros(shape = (len(N)))
    for n in N:
        Q_ln[l][n]=sol.goal_distr[l][N[n]]

Q_ln["ex_wealth"] = np.zeros(shape = (len(N)))
for n in N:
    Q_ln["ex_wealth"][n]=sol.final_exwealth[N[n]]

df_Q_ln = pd.DataFrame(Q_ln)

In [61]:
conf_tg = pd.DataFrame(problem.liabilities.value_tg, columns = problem.liabilities.set, index = N)
conf_lb = pd.DataFrame(problem.liabilities.value_lb, columns = problem.liabilities.set, index = N)
sub_goal_prob = np.logical_and(df_Q_ln[problem.liabilities.set] < conf_tg, df_Q_ln[problem.liabilities.set] >= conf_lb).mean()
fail_goal_prob = (df_Q_ln[problem.liabilities.set] < conf_lb).mean()
print(sub_goal, fail_goal)

{'car': 0.98, 'university': 0.979, 'hawaii': 0.122} {'car': 0.02, 'university': 0.02, 'hawaii': 0.023}
